In [8]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2024,20)
from grid import Grid, cardinal_directions, manhattan_distance
from collections import defaultdict
from solution import *

In [9]:
input_text = """###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""

In [10]:
def get_cheats(grid, walls):
    # cheats should be keyed on (start_point,end_point) and contain a set of points that can be removed
    # start point is the track point ('.', 'S', or 'E') before entering the changed walls
    # end point is the track point ('.', 'S', or 'E') after exiting the changed walls
    # so for each wall, we need to find all the walls that are in the same row or column as
    # the wall as well as the single wall itself
    cheats = {}
    for wall in walls:
        # print(f"Checking wall {wall}")
        start_point = None
        end_point = None
        points_to_remove = set()
        points_to_remove.add(wall)
        
        neighbors = grid.get_neighbors(point=wall, directions=cardinal_directions)
        start_points = set()
        for neighbor in neighbors.values():
            if neighbor not in walls:
                # print(f"  Adding start point {neighbor}")
                start_points.add(neighbor)
        # wall doesn't connect to track
        if len(start_points) == 0:
            # print(f"  Wall {wall} doesn't connect to track")
            continue
    
        for neighbor in neighbors.values():
            if neighbor in walls:
                for start_point in start_points:
                    new_neighbors = grid.get_neighbors(point=neighbor, directions=cardinal_directions)
                    for end_point in neighbors.values():
                        if end_point == start_point or end_point == wall:
                            continue
                        # we can't move through a third wall
                        if end_point in walls:
                            continue
                        # print(f"    Cheat {(start_point, end_point)} goes through {(wall, neighbor)}")
                        cheats[(start_point, end_point)] = (wall, neighbor)
            else:
                end_point = neighbor
                for start_point in start_points:
                    # print(f"    Cheat {(start_point, end_point)} goes through {(wall)}")
                    cheats[(start_point, end_point)] = (wall,)
    return cheats

grid = Grid(input_text, use_overrides=False)
start, goal = None, None
walls = set()
for point, value  in grid.items():
    if value == 'S':
        start = point
        # walls.add(point)
    elif value == 'E':
        goal = point
        # walls.add(point)
    elif value == '#':
        if 0 < point[0] < grid.cfg['max'][0]  and 0 < point[1] < grid.cfg['max'][1]:
            walls.add(point)
cheats = get_cheats(grid, walls)
initial_path = grid.shortest_paths(start, goal)[0]
short_cuts = defaultdict(set)
original_cheats = {}
for cheat, target_walls in cheats.items():
    # print(f"cheat: {cheat}, target_walls: {target_walls}")
    # continue
    # clear cheat points
    for point in target_walls:
        original_cheats[point] = original_cheats.get(point, grid.get_point(point))
        grid.set_point(point, '.')
    cheat_path = grid.shortest_paths(start, goal)[0]
    if len(cheat_path) < len(initial_path):
        # relevant = True
        # for point in cheat:
        #     if point not in cheat_path:
        #         relevant = False
        #         # print(f"{point} not in {cheat_path}")
        #         break
        # if not relevant:
        #     continue
        time_saved = len(initial_path) - len(cheat_path)
        # print(f"Cheating with {cheat} saves {time_saved} steps")
        short_cuts[time_saved].add(cheat)
    # reset cheat points
    for point in target_walls:
        grid.set_point(point, original_cheats[point])

for key in sorted(short_cuts.keys()):
    value = short_cuts[key]
    if len(value) == 1:
        print(f"There is one cheat that saves {key} picoseconds.")
    else:
        print(f"There are {len(value)} cheats that save {key} picoseconds")






There are 72 cheats that save 2 picoseconds
There are 100 cheats that save 4 picoseconds
There are 10 cheats that save 6 picoseconds
There are 21 cheats that save 8 picoseconds
There are 9 cheats that save 10 picoseconds
There are 15 cheats that save 12 picoseconds
There are 2 cheats that save 18 picoseconds
There are 3 cheats that save 20 picoseconds
There are 3 cheats that save 38 picoseconds
There are 6 cheats that save 40 picoseconds
There are 2 cheats that save 64 picoseconds


In [11]:
# for short_cut in short_cuts[64]:
#     counter = 0
#     for point in short_cut:
#         counter += 1
#         grid.set_point(point, str(counter))
#     print(grid)
#     for point in short_cut:
#         grid.set_point(point, original_cheats[point])

In [12]:
# def path_to_linked_list(path):
#     linked_list = {}
#     for i in range(len(path) - 1):
#         linked_list[path[i]] = {
#             'next': path[i+1],
#             'point': path[i],
#             'position': i,
#             'cheat': None
#         }
#     linked_list[path[-1]] = {
#         'next': None,
#         'point': path[-1],
#         'position': len(path) - 1,
#         'cheat': None
#     }
#     return linked_list

# def path_length(start, linked_list):
#     # print(f"path_length({start}, linked_list)")
#     length = -1 # start is not counted
#     current = start
#     while current is not None:
#         # print(f"here current: {current}, next: {linked_list[current]['next']}")
#         if linked_list[current]['cheat'] is not None:
#             length += manhattan_distance(current, linked_list[current]['cheat'])
#             current = linked_list[current]['cheat']
#         else:
#             length += 1
#             current = linked_list[current]['next'] if current in linked_list else None
#     return length

# def apply_cheat(begin, end, linked_list):
#     linked_list[begin]['cheat'] = end  

# def clear_cheats(linked_list):
#     for node in linked_list.values():
#         node['cheat'] = None

# def find_cheats(position, linked_list, grid):
#     # print(f"find_cheats({position}, linked_list)")
#     cheats = []
#     current = linked_list[position]['next']
#     while current is not None:
#         if is_valid_cheat(position, current, grid):
#             cheats.append(current)
#         current = linked_list[current]['next']
#     return cheats

# def is_valid_cheat(point, cheat, grid):
#     # we need to determine if the cheat is valid
#     # rules: cheat must have a manhattan distance of 2 or 3 from point,
#     # and only pass through walls
    
#     # Check if the manhattan distance is 2 or 3
#     distance = manhattan_distance(point, cheat)
#     if distance not in [2, 3]:
#         return False
    
#     # Check if the path between point and cheat only passes through walls
#     x1, y1 = point
#     x2, y2 = cheat
    
#     if x1 == x2:  # Same row
#         for y in range(min(y1, y2) + 1, max(y1, y2)):
#             if grid.get_point((x1, y)) != '#':
#                 return False
#     elif y1 == y2:  # Same column
#         for x in range(min(x1, x2) + 1, max(x1, x2)):
#             if grid.get_point((x, y1)) != '#':
#                 return False
#     else:  # Diagonal
#         if distance == 2:
#             if grid.get_point((x1, y2)) != '#' or grid.get_point((x2, y1)) != '#':
#                 return False
#         elif distance == 3:
#             if abs(x1 - x2) == 1:
#                 if grid.get_point((x1, y2)) != '#' or grid.get_point((x2, y1)) != '#':
#                     return False
#             else:
#                 if grid.get_point((x1, y2)) != '#' or grid.get_point((x2, y1)) != '#':
#                     return False
#     return True

In [15]:

grid = Grid(input_text, use_overrides=False)
start, goal = None, None
walls = set()
for point, value  in grid.items():
    if value == 'S':
        start = point
        # walls.add(point)
    elif value == 'E':
        goal = point
        # walls.add(point)
    elif value == '#':
        if 0 < point[0] < grid.cfg['max'][0]  and 0 < point[1] < grid.cfg['max'][1]:
            walls.add(point)
initial_path = grid.shortest_paths(start, goal)[0]
path_linked_list = path_to_linked_list(initial_path)
intial_length = path_length(start, path_linked_list)
cheat_stats = defaultdict(set)
target = 1
for point in path_linked_list.keys():
        # print(f"Checking point: {point}, position: {path_linked_list[point]['position']}")
        cheats = find_cheats(point, target, path_linked_list, grid)
        if len(cheats) == 0:
            continue
        for cheat in cheats:
            if not is_valid_cheat(point, cheat, grid):
                continue
            # apply_cheat(point, cheat, path_linked_list)
            # length = path_length(start, path_linked_list)
            savings = path_linked_list[cheat]['position'] - path_linked_list[point]['position'] - manhattan_distance(point, cheat)
            # print(f"Applying cheat: {point} -> {cheat}, length: {savings}")
            if savings > target:
                cheat_stats[savings].add((point, cheat))

for size in sorted(cheat_stats.keys()):
    cheats = cheat_stats[size]
    if len(cheats) == 1:
        print(f"There is one cheat that saves {size} picoseconds.")
    else:
        print(f"There are {len(cheats)} cheats that save {size} picoseconds.")


There are 14 cheats that save 2 picoseconds.
There are 14 cheats that save 4 picoseconds.
There are 2 cheats that save 6 picoseconds.
There are 4 cheats that save 8 picoseconds.
There are 2 cheats that save 10 picoseconds.
There are 3 cheats that save 12 picoseconds.
There is one cheat that saves 20 picoseconds.
There is one cheat that saves 36 picoseconds.
There is one cheat that saves 38 picoseconds.
There is one cheat that saves 40 picoseconds.
There is one cheat that saves 64 picoseconds.


In [14]:
print(grid)

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
